In [1]:
import sys
import re 
import numpy as np 
import pandas as pd
import music21
from glob import glob
import IPython
from tqdm import tqdm
import pickle
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
import tensorflow as tf
tf.test.gpu_device_name()

''

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from music21 import converter, instrument, note, chord, stream

In [0]:
songs = glob('/content/drive/My Drive/Final Project/Music/*.mid')

In [0]:
songs = songs[:2]

In [0]:
def get_notes():
    notes = []
    for file in songs:
        midi = converter.parse(file)
        notes_to_parse = []
        
        try:
            parts = instrument.partitionByInstrument(midi)
        except:
            pass
        if parts:
            notes_to_parse = parts.parts[0].recurse()
        else:
            notes_to_parse = midi.flat.notes
    
        for element in notes_to_parse: 
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif(isinstance(element, chord.Chord)):
                notes.append('.'.join(str(n) for n in element.normalOrder))
    with open('/content/drive/My Drive/Final Project/data/notes_1', 'wb') as filepath:
        pickle.dump(notes, filepath)
    
    return notes

In [0]:
def prepare_sequences(notes, n_vocab): 
    sequence_length = 100

    pitchnames = sorted(set(item for item in notes))
    print('Pitchnames: ',len(pitchnames))

    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i: i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])
    
    n_patterns = len(network_input)
    
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    
    network_input = network_input / float(n_vocab)
    
    network_output = np_utils.to_categorical(network_output)
    
    return (network_input, network_output)

In [0]:
from keras.models import Sequential
from keras.layers import Activation, Dense, LSTM, Dropout, Flatten

def create_network(network_in, n_vocab): 

    model = Sequential()
    model.add(LSTM(128, input_shape=network_in.shape[1:], return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(128, return_sequences=True))
    model.add(Flatten())
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')

    return model

In [0]:
from keras.callbacks import ModelCheckpoint
def train(model, network_input, network_output, epochs): 
    # Save model weights.
    filepath = '/content/drive/My Drive/Final Project/weights.best.music3.hdf5'
    checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=0, save_best_only=True)
    
    model.fit(network_input, network_output, epochs=epochs, batch_size=32, callbacks=[checkpoint])

In [0]:
def train_network():
    epochs = 100
    
    notes = get_notes()
    print('Notes processed')
    
    n_vocab = len(set(notes))
    print('Vocab generated',n_vocab)
    
    network_in, network_out = prepare_sequences(notes, n_vocab)
    print('Input and Output processed')
    
    model = create_network(network_in, n_vocab)
    print('Model created')
    print('Training in progress')
    train(model, network_in, network_out, epochs)
    print('Training completed')

In [0]:
train_network()

Notes processed
Vocab generated 155
Pitchnames:  155
Input and Output processed
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model created
Training in progress
Instructions for updating:
Use tf.cast instead.
Epoch 1/100
5553/5553 [==============================] - 78s 14ms/step - loss: 4.4899
Epoch 2/100
5553/5553 [==============================] - 75s 14ms/step - loss: 4.2351
Epoch 3/100
5553/5553 [==============================] - 75s 13ms/step - loss: 4.1074
Epoch 4/100
5553/5553 [==============================] - 75s 13ms/step - loss: 4.0138
Epoch 5/100
5553/5553 [==============================] - 75s 13ms/step - loss: 3.9204
Epoch 6/100
5553/5553 [==============================] - 74s 13ms/step - loss: 3.8140
Epoch 7/100
5553/5553 [==============================] - 75s 13ms/step - loss: 3.6852
Epoch 8/100
5553/5553 [=========================

In [0]:
def generate():
  
    with open('/content/drive/My Drive/Final Project/data/notes_1', 'rb') as filepath:
        notes = pickle.load(filepath)

    pitchnames = sorted(set(item for item in notes))
    n_vocab = len(set(notes))
    
    print('Initiating music generation process.......')
    
    network_input = get_inputSequences(notes, pitchnames, n_vocab)
    
    sequence_length = 100
    
    n_patterns = len(network_input)
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    
    network_input = network_input / float(n_vocab)
    
    model = create_network(network_input, n_vocab)
    print('Loading Model weights.....')
    model.load_weights('/content/drive/My Drive/Final Project/weights.best.music3.hdf5')
    print('Model Loaded')
    prediction_output = generate_notes(model, network_input, pitchnames, n_vocab)
    create_midi(prediction_output)

In [0]:
def get_inputSequences(notes, pitchnames, n_vocab):
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    sequence_length = 100
    network_input = []
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])

    return (network_input)

In [0]:
def generate_notes(model, network_input, pitchnames, n_vocab):
    start = np.random.randint(0, len(network_input)-1)

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
    
    pattern = network_input[start]
    prediction_output = []
    
    print('Generating notes........')

    for note_index in range(500):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)
        index = np.argmax(prediction)
        result = int_to_note[index]
        
        prediction_output.append(result)
        
        pattern = np.concatenate((pattern, [[index]]))
        pattern = pattern[1:len(pattern)]

    print('Notes are generated...')
    return prediction_output

In [0]:
def create_midi(prediction_output):
    offset = 0
    output_notes = []

    for pattern in prediction_output:
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    
    print('Saving Output file as midi....')

    midi_stream.write('midi', fp='/content/drive/My Drive/Final Project/test_output_5.mid')

In [0]:
generate()